# Module 01 — Mathematical & Programming Foundations## 01-07: Probability & Statistics for ML**Objective:** Build the probabilistic foundation for machine learning —distributions, likelihood, MLE, MAP estimation, and Bayes' theorem —from scratch with visual intuition and numerical experiments.**Prerequisites:** 01-01 (Python, NumPy & Tensor Speed), 01-06 (Linear Algebra for ML)

---## Part 0 — Setup & PrerequisitesProbability is the language of uncertainty, and almost every ML algorithmis either explicitly or implicitly probabilistic. Loss functions arenegative log-likelihoods, regularization is a prior, and generalizationis a statement about population distributions.This notebook covers:- **Probability axioms and rules** — independence, conditioning, marginalization- **Common distributions** — Bernoulli, Binomial, Gaussian, Categorical- **Maximum Likelihood Estimation (MLE)** — fitting distributions to data- **Maximum A Posteriori (MAP)** — MLE with priors (= regularization)- **Bayes' theorem** — updating beliefs with evidence- **Central Limit Theorem** — why Gaussians appear everywhere- **Hypothesis testing basics** — p-values, confidence intervalsWe use synthetic data and the California Housing dataset to ground everyconcept in practical ML applications.**Prerequisites:** 01-01, 01-06 (Linear Algebra for ML)

In [ ]:
# ── Imports ──────────────────────────────────────────────────────────────────
import sys
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
from scipy import stats as sp_stats

from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split

print(f'Python: {sys.version.split()[0]}')
print(f'PyTorch: {torch.__version__}')
print(f'NumPy: {np.__version__}')
if torch.cuda.is_available():
    print(f'CUDA: {torch.version.cuda}')
    print(f'GPU: {torch.cuda.get_device_name(0)}')

In [ ]:
# ── Reproducibility ──────────────────────────────────────────────────────────
import random

SEED = 1103
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

In [ ]:
# ── Configuration ────────────────────────────────────────────────────────────
FIGSIZE = (10, 6)
COLORS = {
    'blue': '#1E88E5',
    'red': '#E53935',
    'green': '#43A047',
    'orange': '#FF9800',
    'purple': '#9C27B0',
    'teal': '#00897B',
    'gray': '#757575',
}
COLOR_LIST = list(COLORS.values())

### Data LoadingWe load the California Housing dataset to demonstrate statistical conceptson real-world data.

In [ ]:
# California Housing
housing = fetch_california_housing()
X_housing = housing.data.astype(np.float64)
y_housing = housing.target.astype(np.float64)
feature_names = housing.feature_names
print(f'California Housing: {X_housing.shape}')
print(f'Target (MedHouseVal): mean={y_housing.mean():.3f}, std={y_housing.std():.3f}')
print(f'Features: {feature_names}')

# Quick distribution check
fig, axes = plt.subplots(1, 3, figsize=(15, 4))
axes[0].hist(y_housing, bins=50, color=COLORS['blue'], alpha=0.7, edgecolor='white')
axes[0].set_xlabel('Median House Value ($100K)')
axes[0].set_ylabel('Count')
axes[0].set_title('Target Distribution')

axes[1].hist(X_housing[:, 0], bins=50, color=COLORS['green'], alpha=0.7, edgecolor='white')
axes[1].set_xlabel('Median Income')
axes[1].set_ylabel('Count')
axes[1].set_title('MedInc Distribution')

axes[2].hist(X_housing[:, 1], bins=50, color=COLORS['orange'], alpha=0.7, edgecolor='white')
axes[2].set_xlabel('House Age')
axes[2].set_ylabel('Count')
axes[2].set_title('HouseAge Distribution')

plt.tight_layout()
plt.show()

---## Part 1 — Probability Foundations from ScratchWe build up from probability axioms to the key distributions andestimation methods used in ML.

### 1.1 Discrete Distributions**Bernoulli** — a single coin flip with probability $p$:$$P(X = 1) = p, \quad P(X = 0) = 1 - p$$**Binomial** — $n$ independent coin flips:$$P(X = k) = \binom{n}{k} p^k (1-p)^{n-k}$$**Categorical** — generalization to $K$ classes:$$P(X = k) = p_k, \quad \sum_{k=1}^{K} p_k = 1$$

In [ ]:
def bernoulli_pmf(k: np.ndarray, p: float) -> np.ndarray:
    """Compute Bernoulli PMF.

    Args:
        k: Array of outcomes (0 or 1).
        p: Probability of success.

    Returns:
        Array of probabilities.
    """
    return p ** k * (1 - p) ** (1 - k)


def binomial_pmf(k: np.ndarray, n: int, p: float) -> np.ndarray:
    """Compute Binomial PMF.

    Args:
        k: Array of number of successes.
        n: Number of trials.
        p: Probability of success per trial.

    Returns:
        Array of probabilities.
    """
    from math import comb
    coeffs = np.array([comb(n, int(ki)) for ki in k])
    return coeffs * p ** k * (1 - p) ** (n - k)


def categorical_sample(
    probs: np.ndarray,
    n_samples: int,
    rng: np.random.RandomState,
) -> np.ndarray:
    """Sample from a categorical distribution.

    Args:
        probs: Probability vector of shape (K,), must sum to 1.
        n_samples: Number of samples to draw.
        rng: Random state for reproducibility.

    Returns:
        Array of sampled class indices of shape (n_samples,).
    """
    assert np.isclose(probs.sum(), 1.0), f'Probabilities must sum to 1, got {probs.sum()}'
    cumsum = np.cumsum(probs)
    uniforms = rng.uniform(0, 1, n_samples)
    return np.searchsorted(cumsum, uniforms)


# Visualize discrete distributions
fig, axes = plt.subplots(1, 3, figsize=(16, 4))

# Bernoulli
for p, color in [(0.3, COLORS['blue']), (0.5, COLORS['green']), (0.8, COLORS['red'])]:
    k = np.array([0, 1])
    axes[0].bar(k + (p - 0.5) * 0.25, bernoulli_pmf(k, p), width=0.2,
                color=color, alpha=0.7, label=f'p={p}')
axes[0].set_xticks([0, 1])
axes[0].set_xlabel('k')
axes[0].set_ylabel('P(X=k)')
axes[0].set_title('Bernoulli Distribution')
axes[0].legend()

# Binomial
n_trials = 20
k_vals = np.arange(0, n_trials + 1)
for p, color in [(0.3, COLORS['blue']), (0.5, COLORS['green']), (0.7, COLORS['red'])]:
    axes[1].plot(k_vals, binomial_pmf(k_vals, n_trials, p), 'o-',
                 color=color, markersize=3, label=f'n={n_trials}, p={p}')
axes[1].set_xlabel('k (successes)')
axes[1].set_ylabel('P(X=k)')
axes[1].set_title('Binomial Distribution')
axes[1].legend(fontsize=9)

# Categorical
rng = np.random.RandomState(SEED)
probs = np.array([0.1, 0.3, 0.15, 0.25, 0.2])
samples = categorical_sample(probs, 5000, rng)
counts = np.bincount(samples, minlength=len(probs))
empirical = counts / counts.sum()
x_pos = np.arange(len(probs))
axes[2].bar(x_pos - 0.15, probs, width=0.3, color=COLORS['blue'], alpha=0.7, label='True')
axes[2].bar(x_pos + 0.15, empirical, width=0.3, color=COLORS['red'], alpha=0.7, label='Empirical (5K)')
axes[2].set_xlabel('Class')
axes[2].set_ylabel('Probability')
axes[2].set_title('Categorical Distribution')
axes[2].legend()

plt.tight_layout()
plt.show()

### 1.2 Continuous DistributionsThe **Gaussian (Normal)** distribution is the most important in ML:$$p(x | \mu, \sigma^2) = \frac{1}{\sqrt{2\pi\sigma^2}} \exp\left(-\frac{(x-\mu)^2}{2\sigma^2}\right)$$The **Multivariate Gaussian** generalizes to $d$ dimensions:$$p(\mathbf{x} | \boldsymbol{\mu}, \boldsymbol{\Sigma}) = \frac{1}{(2\pi)^{d/2} |\boldsymbol{\Sigma}|^{1/2}} \exp\left(-\frac{1}{2}(\mathbf{x}-\boldsymbol{\mu})^T \boldsymbol{\Sigma}^{-1} (\mathbf{x}-\boldsymbol{\mu})\right)$$

In [ ]:
def gaussian_pdf(x: np.ndarray, mu: float, sigma: float) -> np.ndarray:
    """Compute univariate Gaussian PDF.

    Args:
        x: Points at which to evaluate the PDF.
        mu: Mean of the distribution.
        sigma: Standard deviation of the distribution.

    Returns:
        Array of density values.
    """
    coeff = 1.0 / (sigma * np.sqrt(2 * np.pi))
    exponent = -0.5 * ((x - mu) / sigma) ** 2
    return coeff * np.exp(exponent)


def multivariate_gaussian_pdf(
    X: np.ndarray,
    mu: np.ndarray,
    cov: np.ndarray,
) -> np.ndarray:
    """Compute multivariate Gaussian PDF.

    Args:
        X: Data points of shape (n_samples, d).
        mu: Mean vector of shape (d,).
        cov: Covariance matrix of shape (d, d).

    Returns:
        Array of density values of shape (n_samples,).
    """
    d = len(mu)
    diff = X - mu
    cov_inv = np.linalg.inv(cov)
    det = np.linalg.det(cov)

    coeff = 1.0 / (np.sqrt((2 * np.pi) ** d * det))
    exponent = -0.5 * np.sum(diff @ cov_inv * diff, axis=1)
    return coeff * np.exp(exponent)


# Visualize Gaussian distributions
fig, axes = plt.subplots(1, 3, figsize=(16, 4))
x_range = np.linspace(-6, 6, 300)

# Different means
for mu, color in [(-2, COLORS['blue']), (0, COLORS['green']), (2, COLORS['red'])]:
    y = gaussian_pdf(x_range, mu, 1.0)
    axes[0].plot(x_range, y, color=color, linewidth=2, label=f'μ={mu}, σ=1')
axes[0].set_xlabel('x')
axes[0].set_ylabel('p(x)')
axes[0].set_title('Varying Mean')
axes[0].legend(fontsize=9)
axes[0].grid(True, alpha=0.2)

# Different variances
for sigma, color in [(0.5, COLORS['blue']), (1.0, COLORS['green']), (2.0, COLORS['red'])]:
    y = gaussian_pdf(x_range, 0, sigma)
    axes[1].plot(x_range, y, color=color, linewidth=2, label=f'μ=0, σ={sigma}')
axes[1].set_xlabel('x')
axes[1].set_ylabel('p(x)')
axes[1].set_title('Varying Standard Deviation')
axes[1].legend(fontsize=9)
axes[1].grid(True, alpha=0.2)

# Verify against scipy
our_pdf = gaussian_pdf(x_range, 1.0, 2.0)
scipy_pdf = sp_stats.norm.pdf(x_range, 1.0, 2.0)
axes[2].plot(x_range, our_pdf, color=COLORS['blue'], linewidth=2, label='Ours')
axes[2].plot(x_range, scipy_pdf, '--', color=COLORS['red'], linewidth=2, label='SciPy')
axes[2].set_xlabel('x')
axes[2].set_ylabel('p(x)')
axes[2].set_title(f'Verification: max diff = {np.max(np.abs(our_pdf - scipy_pdf)):.2e}')
axes[2].legend()
axes[2].grid(True, alpha=0.2)

plt.tight_layout()
plt.show()

### 1.3 Other Distributions in MLBeyond the Gaussian, several distributions appear frequently:- **Uniform** — uninformative priors, random initialization- **Exponential** — waiting times, learning rate schedules- **Beta** — priors on probabilities (Bayesian classification)- **Poisson** — count data (word frequencies, event counts)

In [ ]:
def demonstrate_other_distributions() -> None:
    """Visualize distributions commonly used in ML."""
    fig, axes = plt.subplots(2, 2, figsize=(13, 10))
    x = np.linspace(0.01, 10, 300)
    x_01 = np.linspace(0.001, 0.999, 300)

    # Uniform
    for a, b, color in [(0, 1, COLORS['blue']), (2, 5, COLORS['green']), (0, 10, COLORS['red'])]:
        y = np.where((x >= a) & (x <= b), 1.0 / (b - a), 0.0)
        axes[0, 0].plot(x, y, color=color, linewidth=2, label=f'U({a},{b})')
    axes[0, 0].set_xlabel('x')
    axes[0, 0].set_ylabel('p(x)')
    axes[0, 0].set_title('Uniform Distribution')
    axes[0, 0].legend()
    axes[0, 0].grid(True, alpha=0.2)

    # Exponential
    for lam, color in [(0.5, COLORS['blue']), (1.0, COLORS['green']), (2.0, COLORS['red'])]:
        y = lam * np.exp(-lam * x)
        axes[0, 1].plot(x, y, color=color, linewidth=2, label=f'λ={lam}')
    axes[0, 1].set_xlabel('x')
    axes[0, 1].set_ylabel('p(x)')
    axes[0, 1].set_title('Exponential Distribution')
    axes[0, 1].legend()
    axes[0, 1].grid(True, alpha=0.2)

    # Beta
    for a, b, color in [(0.5, 0.5, COLORS['blue']), (2, 5, COLORS['green']),
                         (5, 2, COLORS['red']), (2, 2, COLORS['orange'])]:
        y = sp_stats.beta.pdf(x_01, a, b)
        axes[1, 0].plot(x_01, y, color=color, linewidth=2, label=f'α={a}, β={b}')
    axes[1, 0].set_xlabel('x')
    axes[1, 0].set_ylabel('p(x)')
    axes[1, 0].set_title('Beta Distribution')
    axes[1, 0].legend(fontsize=9)
    axes[1, 0].grid(True, alpha=0.2)

    # Poisson
    k_vals = np.arange(0, 20)
    for lam, color in [(1, COLORS['blue']), (4, COLORS['green']), (10, COLORS['red'])]:
        y = sp_stats.poisson.pmf(k_vals, lam)
        axes[1, 1].plot(k_vals, y, 'o-', color=color, markersize=4,
                         linewidth=1.5, label=f'λ={lam}')
    axes[1, 1].set_xlabel('k')
    axes[1, 1].set_ylabel('P(X=k)')
    axes[1, 1].set_title('Poisson Distribution')
    axes[1, 1].legend()
    axes[1, 1].grid(True, alpha=0.2)

    plt.suptitle('Common Distributions in ML', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()


demonstrate_other_distributions()

### 1.4 Maximum Likelihood Estimation (MLE)Given observed data $\mathcal{D} = \{x_1, \ldots, x_n\}$, MLE finds theparameter $\theta$ that maximizes the likelihood:$$\hat{\theta}_{\text{MLE}} = \arg\max_\theta \prod_{i=1}^{n} p(x_i | \theta)$$In practice, we maximize the **log-likelihood** (sum instead of product):$$\hat{\theta}_{\text{MLE}} = \arg\max_\theta \sum_{i=1}^{n} \log p(x_i | \theta)$$**Connection to ML:** Minimizing cross-entropy loss = maximizing log-likelihoodof the data under the model.

In [ ]:
def gaussian_log_likelihood(
    data: np.ndarray,
    mu: float,
    sigma: float,
) -> float:
    """Compute log-likelihood of data under a Gaussian model.

    Args:
        data: Observed values of shape (n,).
        mu: Mean parameter.
        sigma: Standard deviation parameter.

    Returns:
        Total log-likelihood value.
    """
    n = len(data)
    ll = -0.5 * n * np.log(2 * np.pi)
    ll -= n * np.log(sigma)
    ll -= 0.5 * np.sum((data - mu) ** 2) / sigma ** 2
    return ll


def mle_gaussian(data: np.ndarray) -> tuple[float, float]:
    """Compute MLE for Gaussian parameters (closed-form).

    For a Gaussian, the MLE solutions are:
    - μ_MLE = sample mean
    - σ²_MLE = sample variance (biased, dividing by n)

    Args:
        data: Observed values of shape (n,).

    Returns:
        Tuple of (mu_mle, sigma_mle).
    """
    mu_mle = np.mean(data)
    sigma_mle = np.sqrt(np.mean((data - mu_mle) ** 2))  # Biased (MLE)
    return mu_mle, sigma_mle


# Generate data from a known Gaussian
rng = np.random.RandomState(SEED)
true_mu, true_sigma = 3.0, 1.5
data_mle = rng.normal(true_mu, true_sigma, size=500)

mu_hat, sigma_hat = mle_gaussian(data_mle)
print(f'True parameters:  μ={true_mu}, σ={true_sigma}')
print(f'MLE estimates:    μ̂={mu_hat:.4f}, σ̂={sigma_hat:.4f}')
print(f'Log-likelihood at MLE: {gaussian_log_likelihood(data_mle, mu_hat, sigma_hat):.4f}')
print(f'Log-likelihood at true: {gaussian_log_likelihood(data_mle, true_mu, true_sigma):.4f}')

Let's visualize how the log-likelihood surface looks and how MLE findsthe peak.

In [ ]:
def visualize_mle_surface(data: np.ndarray, true_mu: float, true_sigma: float) -> None:
    """Visualize the log-likelihood surface for Gaussian MLE.

    Args:
        data: Observed data.
        true_mu: True mean.
        true_sigma: True standard deviation.
    """
    mu_hat, sigma_hat = mle_gaussian(data)

    fig, axes = plt.subplots(1, 3, figsize=(18, 5))

    # 1D slice: LL vs mu (sigma fixed at MLE)
    mu_range = np.linspace(true_mu - 2, true_mu + 2, 200)
    ll_mu = [gaussian_log_likelihood(data, m, sigma_hat) for m in mu_range]
    axes[0].plot(mu_range, ll_mu, color=COLORS['blue'], linewidth=2)
    axes[0].axvline(mu_hat, color=COLORS['red'], linestyle='--',
                     label=f'μ̂_MLE = {mu_hat:.3f}')
    axes[0].axvline(true_mu, color=COLORS['green'], linestyle=':',
                     label=f'μ_true = {true_mu}')
    axes[0].set_xlabel('μ')
    axes[0].set_ylabel('Log-likelihood')
    axes[0].set_title('LL vs μ (σ fixed at MLE)')
    axes[0].legend(fontsize=9)
    axes[0].grid(True, alpha=0.2)

    # 1D slice: LL vs sigma (mu fixed at MLE)
    sigma_range = np.linspace(0.5, 3.5, 200)
    ll_sigma = [gaussian_log_likelihood(data, mu_hat, s) for s in sigma_range]
    axes[1].plot(sigma_range, ll_sigma, color=COLORS['blue'], linewidth=2)
    axes[1].axvline(sigma_hat, color=COLORS['red'], linestyle='--',
                     label=f'σ̂_MLE = {sigma_hat:.3f}')
    axes[1].axvline(true_sigma, color=COLORS['green'], linestyle=':',
                     label=f'σ_true = {true_sigma}')
    axes[1].set_xlabel('σ')
    axes[1].set_ylabel('Log-likelihood')
    axes[1].set_title('LL vs σ (μ fixed at MLE)')
    axes[1].legend(fontsize=9)
    axes[1].grid(True, alpha=0.2)

    # 2D contour
    mu_grid = np.linspace(true_mu - 1.5, true_mu + 1.5, 100)
    sigma_grid = np.linspace(0.8, 2.5, 100)
    ll_surface = np.zeros((len(sigma_grid), len(mu_grid)))
    for i, s in enumerate(sigma_grid):
        for j, m in enumerate(mu_grid):
            ll_surface[i, j] = gaussian_log_likelihood(data, m, s)

    axes[2].contourf(mu_grid, sigma_grid, ll_surface, levels=30, cmap='viridis')
    axes[2].plot(mu_hat, sigma_hat, 'r*', markersize=15, label='MLE')
    axes[2].plot(true_mu, true_sigma, 'g^', markersize=10, label='True')
    axes[2].set_xlabel('μ')
    axes[2].set_ylabel('σ')
    axes[2].set_title('Log-Likelihood Surface')
    axes[2].legend()

    plt.suptitle('Maximum Likelihood Estimation', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()


visualize_mle_surface(data_mle, true_mu, true_sigma)

### 1.5 MLE for Bernoulli (Classification Connection)For binary classification, the model outputs $p(y=1|\mathbf{x})$.The MLE for Bernoulli is simply the sample proportion:$$\hat{p}_{\text{MLE}} = \frac{1}{n}\sum_{i=1}^{n} y_i$$The negative log-likelihood is the **binary cross-entropy loss**:$$\mathcal{L} = -\frac{1}{n}\sum_i \left[y_i \log(p_i) + (1-y_i)\log(1-p_i)\right]$$

In [ ]:
def bernoulli_nll(labels: np.ndarray, predicted_probs: np.ndarray) -> float:
    """Compute binary cross-entropy (negative log-likelihood for Bernoulli).

    Args:
        labels: True binary labels of shape (n,).
        predicted_probs: Predicted probabilities of shape (n,).

    Returns:
        Average negative log-likelihood (binary cross-entropy).
    """
    eps = 1e-15  # Numerical stability
    predicted_probs = np.clip(predicted_probs, eps, 1 - eps)
    nll = -(labels * np.log(predicted_probs) + (1 - labels) * np.log(1 - predicted_probs))
    return nll.mean()


# Demonstrate connection: MLE of coin flips
rng = np.random.RandomState(SEED)
true_p = 0.7
coin_flips = rng.binomial(1, true_p, size=1000)

p_hat = coin_flips.mean()  # MLE
print(f'Coin flips: n={len(coin_flips)}, sum={coin_flips.sum()}')
print(f'True p = {true_p}')
print(f'MLE p̂ = {p_hat:.4f}')
print()

# BCE at different p values
p_range = np.linspace(0.01, 0.99, 200)
bce_values = [bernoulli_nll(coin_flips, np.full(len(coin_flips), p)) for p in p_range]

fig, ax = plt.subplots(figsize=(8, 5))
ax.plot(p_range, bce_values, color=COLORS['blue'], linewidth=2)
ax.axvline(p_hat, color=COLORS['red'], linestyle='--', label=f'MLE p̂ = {p_hat:.4f}')
ax.axvline(true_p, color=COLORS['green'], linestyle=':', label=f'True p = {true_p}')
ax.set_xlabel('p')
ax.set_ylabel('Binary Cross-Entropy')
ax.set_title('BCE vs p — Minimum = MLE')
ax.legend()
ax.grid(True, alpha=0.2)
plt.tight_layout()
plt.show()

print(f'BCE at MLE:  {bernoulli_nll(coin_flips, np.full(len(coin_flips), p_hat)):.6f}')
print(f'BCE at true: {bernoulli_nll(coin_flips, np.full(len(coin_flips), true_p)):.6f}')

### 1.6 Maximum A Posteriori (MAP) EstimationMAP adds a **prior** distribution $p(\theta)$ to MLE:$$\hat{\theta}_{\text{MAP}} = \arg\max_\theta \left[\sum_i \log p(x_i | \theta) + \log p(\theta)\right]$$**The key insight:** MAP = MLE + regularization.- Gaussian prior on weights → L2 regularization (Ridge)- Laplace prior on weights → L1 regularization (Lasso)

In [ ]:
def map_gaussian_with_prior(
    data: np.ndarray,
    prior_mu: float,
    prior_sigma: float,
    data_sigma: float,
) -> float:
    """Compute MAP estimate for Gaussian mean with Gaussian prior.

    Closed-form: MAP = weighted average of prior mean and MLE.

    Args:
        data: Observed values.
        prior_mu: Prior mean for the parameter.
        prior_sigma: Prior standard deviation (confidence in prior).
        data_sigma: Known data standard deviation.

    Returns:
        MAP estimate of the mean.
    """
    n = len(data)
    data_mean = np.mean(data)
    prior_precision = 1.0 / prior_sigma ** 2
    data_precision = n / data_sigma ** 2

    # MAP is a precision-weighted average
    map_mu = (prior_precision * prior_mu + data_precision * data_mean) / (
        prior_precision + data_precision
    )
    return map_mu


def visualize_map_vs_mle() -> None:
    """Show how MAP interpolates between prior and MLE."""
    rng = np.random.RandomState(SEED)
    true_mu = 5.0
    data_sigma = 2.0
    prior_mu = 0.0  # Prior centered at 0

    sample_sizes = [1, 5, 10, 50, 200, 1000]
    prior_sigmas = [1.0, 2.0, 5.0]

    fig, axes = plt.subplots(1, 2, figsize=(14, 5))

    # Effect of sample size
    for ps, color in zip(prior_sigmas, [COLORS['blue'], COLORS['green'], COLORS['red']]):
        map_estimates = []
        mle_estimates = []
        for n in sample_sizes:
            data = rng.normal(true_mu, data_sigma, n)
            map_est = map_gaussian_with_prior(data, prior_mu, ps, data_sigma)
            mle_est = np.mean(data)
            map_estimates.append(map_est)
            mle_estimates.append(mle_est)

        axes[0].plot(sample_sizes, map_estimates, 'o-', color=color,
                      linewidth=2, label=f'MAP (σ_prior={ps})')

    axes[0].plot(sample_sizes, mle_estimates, 's--', color=COLORS['gray'],
                  linewidth=1.5, label='MLE (no prior)')
    axes[0].axhline(true_mu, color='black', linestyle=':', alpha=0.5, label=f'True μ={true_mu}')
    axes[0].axhline(prior_mu, color=COLORS['orange'], linestyle=':', alpha=0.5, label=f'Prior μ={prior_mu}')
    axes[0].set_xlabel('Sample Size (n)')
    axes[0].set_ylabel('Estimated μ')
    axes[0].set_title('MAP vs MLE: Effect of Sample Size')
    axes[0].legend(fontsize=8)
    axes[0].set_xscale('log')
    axes[0].grid(True, alpha=0.2)

    # Regularization connection
    lambda_range = np.linspace(0, 5, 200)
    data_small = rng.normal(true_mu, data_sigma, 10)
    x_bar = np.mean(data_small)
    n = len(data_small)

    # Ridge regression analogy: θ_MAP = (X^TX + λI)^{-1} X^Ty
    # For 1D: θ_MAP = n·x_bar / (n + λ·σ²) when prior_mu = 0
    map_lambda = [n * x_bar / (n + lam * data_sigma ** 2) for lam in lambda_range]

    axes[1].plot(lambda_range, map_lambda, color=COLORS['blue'], linewidth=2)
    axes[1].axhline(x_bar, color=COLORS['green'], linestyle='--',
                     label=f'MLE = {x_bar:.2f}')
    axes[1].axhline(0, color=COLORS['red'], linestyle=':', label='Prior (λ→∞)')
    axes[1].set_xlabel('Regularization Strength (λ = 1/σ²_prior)')
    axes[1].set_ylabel('MAP Estimate')
    axes[1].set_title('MAP ↔ Regularization Connection')
    axes[1].legend()
    axes[1].grid(True, alpha=0.2)

    plt.tight_layout()
    plt.show()

    print('Key insight:')
    print('  - MAP with Gaussian prior = Ridge regression (L2 penalty)')
    print('  - MAP with Laplace prior = Lasso (L1 penalty)')
    print('  - More data → MAP approaches MLE (data overwhelms prior)')


visualize_map_vs_mle()

### 1.7 Bayes' TheoremBayes' theorem relates conditional probabilities:$$P(\theta | \mathcal{D}) = \frac{P(\mathcal{D} | \theta) \cdot P(\theta)}{P(\mathcal{D})}$$- $P(\theta | \mathcal{D})$ — **posterior** (updated belief after seeing data)- $P(\mathcal{D} | \theta)$ — **likelihood** (how probable the data is under $\theta$)- $P(\theta)$ — **prior** (belief before seeing data)- $P(\mathcal{D})$ — **evidence** (normalization constant)MAP finds the **mode** of the posterior. Full Bayesian inference computesthe **entire posterior distribution**.

In [ ]:
def bayesian_coin_inference() -> None:
    """Demonstrate Bayesian updating for coin fairness."""
    rng = np.random.RandomState(SEED)
    true_p = 0.65
    all_flips = rng.binomial(1, true_p, size=200)

    # Prior: Beta(2, 2) — mild belief that coin is fair
    alpha_prior, beta_prior = 2, 2
    p_grid = np.linspace(0, 1, 500)

    fig, axes = plt.subplots(2, 3, figsize=(16, 9))
    axes = axes.ravel()
    checkpoints = [0, 1, 5, 20, 50, 200]

    for idx, n_obs in enumerate(checkpoints):
        observed = all_flips[:n_obs]
        heads = observed.sum() if n_obs > 0 else 0
        tails = n_obs - heads

        # Posterior: Beta(alpha + heads, beta + tails)
        alpha_post = alpha_prior + heads
        beta_post = beta_prior + tails
        posterior = sp_stats.beta.pdf(p_grid, alpha_post, beta_post)
        prior = sp_stats.beta.pdf(p_grid, alpha_prior, beta_prior)

        # Plot
        if n_obs == 0:
            axes[idx].plot(p_grid, prior, color=COLORS['blue'], linewidth=2, label='Prior')
        else:
            axes[idx].fill_between(p_grid, posterior, alpha=0.3, color=COLORS['blue'])
            axes[idx].plot(p_grid, posterior, color=COLORS['blue'], linewidth=2, label='Posterior')
            axes[idx].plot(p_grid, prior, '--', color=COLORS['gray'],
                            linewidth=1, label='Prior', alpha=0.5)

        axes[idx].axvline(true_p, color=COLORS['red'], linestyle=':',
                           label=f'True p={true_p}', alpha=0.7)

        if n_obs > 0:
            mle_p = heads / n_obs
            map_p = (alpha_post - 1) / (alpha_post + beta_post - 2)
            axes[idx].axvline(mle_p, color=COLORS['green'], linestyle='--',
                               label=f'MLE={mle_p:.2f}', alpha=0.7)

        axes[idx].set_xlabel('p')
        axes[idx].set_ylabel('Density')
        title = f'n={n_obs}' if n_obs > 0 else 'Prior (n=0)'
        if n_obs > 0:
            title += f' | H={heads}, T={tails}'
        axes[idx].set_title(title)
        axes[idx].legend(fontsize=7)
        axes[idx].set_xlim(0, 1)

    plt.suptitle('Bayesian Coin Inference: Posterior Update', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()


bayesian_coin_inference()

### 1.8 Multivariate Gaussian VisualizationThe multivariate Gaussian is the cornerstone of many ML algorithms(Gaussian Mixture Models, Gaussian Processes, Kalman Filters). Let'svisualize how the covariance matrix shapes the distribution.

In [ ]:
def visualize_multivariate_gaussians() -> None:
    """Visualize 2D Gaussians with different covariance matrices."""
    configs = [
        ('Spherical', np.array([[1, 0], [0, 1]]), np.array([0, 0])),
        ('Diagonal', np.array([[3, 0], [0, 0.5]]), np.array([0, 0])),
        ('Correlated', np.array([[2, 1.5], [1.5, 2]]), np.array([0, 0])),
        ('Anti-correlated', np.array([[2, -1.5], [-1.5, 2]]), np.array([0, 0])),
    ]

    fig, axes = plt.subplots(2, 2, figsize=(12, 10))
    axes = axes.ravel()

    x_grid = np.linspace(-5, 5, 100)
    y_grid = np.linspace(-5, 5, 100)
    xx, yy = np.meshgrid(x_grid, y_grid)
    grid_points = np.column_stack([xx.ravel(), yy.ravel()])

    for idx, (name, cov, mu) in enumerate(configs):
        # Compute PDF on grid
        pdf_vals = multivariate_gaussian_pdf(grid_points, mu, cov)
        pdf_grid = pdf_vals.reshape(xx.shape)

        # Contour plot
        axes[idx].contourf(xx, yy, pdf_grid, levels=20, cmap='viridis', alpha=0.8)
        axes[idx].contour(xx, yy, pdf_grid, levels=5, colors='white', linewidths=0.5)

        # Draw samples
        rng_mv = np.random.RandomState(SEED)
        samples = rng_mv.multivariate_normal(mu, cov, 200)
        axes[idx].scatter(samples[:, 0], samples[:, 1], s=5, color='white', alpha=0.5)

        # Show eigenvectors of covariance
        eigvals, eigvecs = np.linalg.eigh(cov)
        for i in range(2):
            vec = eigvecs[:, i] * np.sqrt(eigvals[i]) * 2
            axes[idx].annotate('', xy=mu + vec, xytext=mu,
                                arrowprops=dict(arrowstyle='->', color=COLORS['red'], lw=2))

        axes[idx].set_xlabel('$x_1$')
        axes[idx].set_ylabel('$x_2$')
        axes[idx].set_title(f'{name}\nΣ = {cov.tolist()}')
        axes[idx].set_aspect('equal')
        axes[idx].set_xlim(-5, 5)
        axes[idx].set_ylim(-5, 5)

    plt.suptitle('Multivariate Gaussian: Effect of Covariance', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()


visualize_multivariate_gaussians()

---## Part 2 — Putting It All Together: Statistical ToolkitLet's assemble our probability primitives into a reusable toolkit forstatistical analysis in ML.

In [ ]:
class StatisticalToolkit:
    """Reusable statistical analysis toolkit for ML.

    Provides methods for distribution fitting, hypothesis testing,
    confidence intervals, and distribution comparison.

    Attributes:
        data: The data array being analyzed.
        n: Number of samples.
    """

    def __init__(self, data: np.ndarray) -> None:
        """Initialize with data.

        Args:
            data: 1D array of observations.
        """
        self.data = data
        self.n = len(data)

    def descriptive_stats(self) -> pd.DataFrame:
        """Compute comprehensive descriptive statistics.

        Returns:
            DataFrame with statistic names and values.
        """
        stats_dict = {
            'Count': self.n,
            'Mean': np.mean(self.data),
            'Std': np.std(self.data, ddof=1),
            'Min': np.min(self.data),
            'Q1 (25%)': np.percentile(self.data, 25),
            'Median': np.median(self.data),
            'Q3 (75%)': np.percentile(self.data, 75),
            'Max': np.max(self.data),
            'Skewness': sp_stats.skew(self.data),
            'Kurtosis': sp_stats.kurtosis(self.data),
            'IQR': np.percentile(self.data, 75) - np.percentile(self.data, 25),
        }
        return pd.DataFrame({
            'Statistic': list(stats_dict.keys()),
            'Value': [f'{v:.4f}' if isinstance(v, float) else str(v)
                      for v in stats_dict.values()],
        })

    def confidence_interval(
        self,
        confidence: float = 0.95,
    ) -> tuple[float, float]:
        """Compute confidence interval for the mean.

        Uses the t-distribution for small samples.

        Args:
            confidence: Confidence level (e.g., 0.95 for 95%).

        Returns:
            Tuple of (lower_bound, upper_bound).
        """
        mean = np.mean(self.data)
        se = sp_stats.sem(self.data)
        alpha = 1 - confidence
        t_crit = sp_stats.t.ppf(1 - alpha / 2, df=self.n - 1)
        return (mean - t_crit * se, mean + t_crit * se)

    def normality_test(self) -> dict[str, float]:
        """Test if data follows a normal distribution.

        Uses the Shapiro-Wilk test (best for n < 5000).

        Returns:
            Dictionary with test statistic and p-value.
        """
        sample = self.data[:5000] if self.n > 5000 else self.data
        stat, p_value = sp_stats.shapiro(sample)
        return {'statistic': stat, 'p_value': p_value}

    def fit_gaussian(self) -> tuple[float, float]:
        """Fit Gaussian distribution via MLE.

        Returns:
            Tuple of (mu_mle, sigma_mle).
        """
        return mle_gaussian(self.data)


# Sanity check
toolkit = StatisticalToolkit(y_housing)
print('=== California Housing Target — Descriptive Statistics ===')
print(toolkit.descriptive_stats().to_string(index=False))
print()

ci_low, ci_high = toolkit.confidence_interval(0.95)
print(f'95% CI for mean: [{ci_low:.4f}, {ci_high:.4f}]')

norm_test = toolkit.normality_test()
print(f'\nNormality test: statistic={norm_test["statistic"]:.4f}, '
      f'p-value={norm_test["p_value"]:.4e}')
print(f'Normal? {"Yes" if norm_test["p_value"] > 0.05 else "No"} (at α=0.05)')

---## Part 3 — Application: Statistical Analysis of Housing DataLet's apply our tools to analyze the California Housing datasetand demonstrate how probability connects to ML.

### 3.1 Central Limit Theorem DemonstrationThe CLT says that sample means converge to a Gaussian, regardlessof the underlying distribution. This is why Gaussian assumptionswork surprisingly well in practice.

In [ ]:
def demonstrate_clt() -> None:
    """Demonstrate the Central Limit Theorem with different source distributions."""
    rng = np.random.RandomState(SEED)

    distributions = {
        'Exponential (λ=1)': lambda n: rng.exponential(1.0, n),
        'Uniform (0, 1)': lambda n: rng.uniform(0, 1, n),
        'Bernoulli (p=0.3)': lambda n: rng.binomial(1, 0.3, n),
    }

    sample_sizes = [1, 5, 30, 100]
    n_simulations = 5000

    fig, axes = plt.subplots(len(distributions), len(sample_sizes),
                              figsize=(16, 10), sharex=False)

    for row, (dist_name, sampler) in enumerate(distributions.items()):
        for col, n in enumerate(sample_sizes):
            means = [sampler(n).mean() for _ in range(n_simulations)]

            axes[row, col].hist(means, bins=40, density=True,
                                 color=COLOR_LIST[row], alpha=0.7, edgecolor='white')

            # Overlay theoretical Gaussian
            grand_mean = np.mean(means)
            grand_std = np.std(means)
            x_plot = np.linspace(grand_mean - 4 * grand_std, grand_mean + 4 * grand_std, 200)
            axes[row, col].plot(x_plot, gaussian_pdf(x_plot, grand_mean, grand_std),
                                 'k--', linewidth=1.5, label='Gaussian fit')

            if row == 0:
                axes[row, col].set_title(f'n = {n}', fontsize=11)
            if col == 0:
                axes[row, col].set_ylabel(dist_name, fontsize=10)

    plt.suptitle('Central Limit Theorem: Sample Mean Distribution', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()

    print('Observation: Even n=30 gives approximately Gaussian sample means,')
    print('regardless of the source distribution.')


demonstrate_clt()

### 3.2 Law of Large NumbersThe LLN states that the sample mean converges to the true mean as$n \to \infty$. This is why SGD works — each mini-batch gradientis a noisy estimate that averages out over many steps.

In [ ]:
def demonstrate_law_of_large_numbers() -> None:
    """Visualize how running averages converge to the true mean."""
    rng_lln = np.random.RandomState(SEED)

    distributions = {
        'Gaussian (μ=5)': (rng_lln.normal, {'loc': 5.0, 'scale': 2.0}),
        'Exponential (λ=0.5)': (rng_lln.exponential, {'scale': 2.0}),
        'Uniform (0, 10)': (rng_lln.uniform, {'low': 0, 'high': 10}),
    }
    true_means = [5.0, 2.0, 5.0]

    fig, axes = plt.subplots(1, 3, figsize=(16, 4))
    n_max = 5000

    for idx, ((name, (sampler, params)), true_mean) in enumerate(
        zip(distributions.items(), true_means)
    ):
        samples = sampler(size=n_max, **params)
        running_mean = np.cumsum(samples) / np.arange(1, n_max + 1)

        axes[idx].plot(running_mean, color=COLOR_LIST[idx], linewidth=1, alpha=0.8)
        axes[idx].axhline(true_mean, color=COLORS['red'], linestyle='--',
                           label=f'True μ = {true_mean}', linewidth=1.5)
        axes[idx].fill_between(
            range(n_max),
            true_mean - 2 * np.std(samples) / np.sqrt(np.arange(1, n_max + 1)),
            true_mean + 2 * np.std(samples) / np.sqrt(np.arange(1, n_max + 1)),
            alpha=0.1, color=COLORS['red'],
        )
        axes[idx].set_xlabel('Number of Samples')
        axes[idx].set_ylabel('Running Mean')
        axes[idx].set_title(name)
        axes[idx].legend(fontsize=9)
        axes[idx].grid(True, alpha=0.2)

    plt.suptitle('Law of Large Numbers: Running Mean → True Mean', fontsize=13, fontweight='bold')
    plt.tight_layout()
    plt.show()


demonstrate_law_of_large_numbers()

### 3.3 Distribution Fitting on Real DataLet's fit different distributions to housing features and comparetheir log-likelihoods.

In [ ]:
def fit_distributions_to_housing() -> None:
    """Fit multiple distributions to housing features and compare."""
    features_to_test = {
        'MedInc': X_housing[:, 0],
        'HouseAge': X_housing[:, 1],
        'MedHouseVal': y_housing,
    }

    fig, axes = plt.subplots(1, 3, figsize=(18, 5))

    for idx, (name, data) in enumerate(features_to_test.items()):
        axes[idx].hist(data, bins=50, density=True, color=COLORS['gray'],
                        alpha=0.5, edgecolor='white', label='Data')

        x_range = np.linspace(data.min(), data.max(), 200)

        # Fit Gaussian
        mu, sigma = mle_gaussian(data)
        axes[idx].plot(x_range, gaussian_pdf(x_range, mu, sigma),
                        color=COLORS['blue'], linewidth=2, label=f'Gaussian (μ={mu:.2f})')

        # Fit Log-normal (for positive data)
        if data.min() > 0:
            log_data = np.log(data)
            mu_ln, sigma_ln = mle_gaussian(log_data)
            lognorm_pdf = gaussian_pdf(np.log(x_range), mu_ln, sigma_ln) / x_range
            axes[idx].plot(x_range, lognorm_pdf,
                            color=COLORS['red'], linewidth=2, label='Log-Normal')

        axes[idx].set_xlabel(name)
        axes[idx].set_ylabel('Density')
        axes[idx].set_title(f'{name} — Distribution Fit')
        axes[idx].legend(fontsize=9)
        axes[idx].grid(True, alpha=0.2)

    plt.tight_layout()
    plt.show()


fit_distributions_to_housing()

### 3.3 Bootstrap Confidence IntervalsThe bootstrap is a powerful non-parametric method for estimatinguncertainty. Instead of assuming a distribution, it resamples the datawith replacement to build an empirical sampling distribution.

In [ ]:
def bootstrap_confidence_interval(
    data: np.ndarray,
    statistic_fn: callable,
    n_bootstrap: int = 5000,
    confidence: float = 0.95,
) -> tuple[float, float, np.ndarray]:
    """Compute bootstrap confidence interval for any statistic.

    Args:
        data: Input data array.
        statistic_fn: Function that computes the statistic from data.
        n_bootstrap: Number of bootstrap samples.
        confidence: Confidence level.

    Returns:
        Tuple of (lower, upper, bootstrap_distribution).
    """
    rng_bs = np.random.RandomState(SEED)
    n = len(data)
    bootstrap_stats = np.zeros(n_bootstrap)

    for i in range(n_bootstrap):
        resample = data[rng_bs.choice(n, size=n, replace=True)]
        bootstrap_stats[i] = statistic_fn(resample)

    alpha = 1 - confidence
    lower = np.percentile(bootstrap_stats, 100 * alpha / 2)
    upper = np.percentile(bootstrap_stats, 100 * (1 - alpha / 2))
    return lower, upper, bootstrap_stats


# Bootstrap CI for mean, median, and std of housing prices
stats_to_test = {
    'Mean': np.mean,
    'Median': np.median,
    'Std': lambda x: np.std(x, ddof=1),
}

fig, axes = plt.subplots(1, 3, figsize=(16, 4))

for idx, (name, fn) in enumerate(stats_to_test.items()):
    low, high, boot_dist = bootstrap_confidence_interval(y_housing, fn)
    point_est = fn(y_housing)

    axes[idx].hist(boot_dist, bins=50, color=COLOR_LIST[idx], alpha=0.7, edgecolor='white')
    axes[idx].axvline(point_est, color='black', linestyle='-', linewidth=2, label=f'{name}={point_est:.4f}')
    axes[idx].axvline(low, color=COLORS['red'], linestyle='--', label=f'95% CI: [{low:.4f}')
    axes[idx].axvline(high, color=COLORS['red'], linestyle='--', label=f'         {high:.4f}]')
    axes[idx].set_xlabel(name)
    axes[idx].set_ylabel('Count')
    axes[idx].set_title(f'Bootstrap {name}')
    axes[idx].legend(fontsize=8)

plt.suptitle('Bootstrap Confidence Intervals — MedHouseVal', fontsize=13)
plt.tight_layout()
plt.show()

### 3.4 Covariance and CorrelationThe covariance matrix captures linear relationships between features.Correlation normalizes by standard deviations for interpretability.

In [ ]:
def compute_covariance_from_scratch(X: np.ndarray) -> np.ndarray:
    """Compute covariance matrix from scratch.

    Args:
        X: Data matrix of shape (n_samples, n_features).

    Returns:
        Covariance matrix of shape (n_features, n_features).
    """
    n = X.shape[0]
    X_centered = X - X.mean(axis=0)
    return (X_centered.T @ X_centered) / (n - 1)


def compute_correlation_from_scratch(X: np.ndarray) -> np.ndarray:
    """Compute Pearson correlation matrix from scratch.

    Args:
        X: Data matrix of shape (n_samples, n_features).

    Returns:
        Correlation matrix of shape (n_features, n_features).
    """
    cov = compute_covariance_from_scratch(X)
    stds = np.sqrt(np.diag(cov))
    return cov / np.outer(stds, stds)


# Compute and verify
our_cov = compute_covariance_from_scratch(X_housing)
np_cov = np.cov(X_housing.T)
assert our_cov.shape == (8, 8), f'Expected (8, 8), got {our_cov.shape}'
print(f'Covariance match: {np.allclose(our_cov, np_cov)}')

our_corr = compute_correlation_from_scratch(X_housing)
np_corr = np.corrcoef(X_housing.T)
print(f'Correlation match: {np.allclose(our_corr, np_corr)}')

# Visualize correlation matrix
fig, ax = plt.subplots(figsize=(9, 7))
im = ax.imshow(our_corr, cmap='coolwarm', vmin=-1, vmax=1)
ax.set_xticks(range(8))
ax.set_xticklabels(feature_names, rotation=45, ha='right')
ax.set_yticks(range(8))
ax.set_yticklabels(feature_names)
for i in range(8):
    for j in range(8):
        ax.text(j, i, f'{our_corr[i,j]:.2f}', ha='center', va='center', fontsize=8)
plt.colorbar(im, ax=ax, shrink=0.8)
ax.set_title('Feature Correlation Matrix — California Housing')
plt.tight_layout()
plt.show()

---## Part 4 — Evaluation & AnalysisLet's evaluate our statistical tools, examine hypothesis testing,and build a comprehensive reference.

### 4.1 MLE Convergence: Effect of Sample SizeHow quickly does MLE converge to the true parameters? Let's measurethis empirically.

In [ ]:
def analyze_mle_convergence() -> None:
    """Show how MLE estimates improve with more data."""
    rng = np.random.RandomState(SEED)
    true_mu, true_sigma = 3.0, 1.5

    sample_sizes = [5, 10, 25, 50, 100, 250, 500, 1000, 5000]
    n_trials = 200

    mu_errors = []
    sigma_errors = []

    for n in sample_sizes:
        mu_errs = []
        sigma_errs = []
        for _ in range(n_trials):
            data = rng.normal(true_mu, true_sigma, n)
            mu_hat, sigma_hat = mle_gaussian(data)
            mu_errs.append(abs(mu_hat - true_mu))
            sigma_errs.append(abs(sigma_hat - true_sigma))
        mu_errors.append(np.mean(mu_errs))
        sigma_errors.append(np.mean(sigma_errs))

    fig, axes = plt.subplots(1, 2, figsize=(14, 5))

    axes[0].loglog(sample_sizes, mu_errors, 'o-', color=COLORS['blue'], linewidth=2, label='|μ̂ - μ|')
    # Theoretical: error ∝ 1/√n
    theoretical = mu_errors[0] * np.sqrt(sample_sizes[0]) / np.sqrt(sample_sizes)
    axes[0].loglog(sample_sizes, theoretical, '--', color=COLORS['red'], label='O(1/√n)')
    axes[0].set_xlabel('Sample Size (n)')
    axes[0].set_ylabel('Mean Absolute Error')
    axes[0].set_title('MLE Convergence: Mean')
    axes[0].legend()
    axes[0].grid(True, alpha=0.2)

    axes[1].loglog(sample_sizes, sigma_errors, 'o-', color=COLORS['green'], linewidth=2, label='|σ̂ - σ|')
    theoretical_s = sigma_errors[0] * np.sqrt(sample_sizes[0]) / np.sqrt(sample_sizes)
    axes[1].loglog(sample_sizes, theoretical_s, '--', color=COLORS['red'], label='O(1/√n)')
    axes[1].set_xlabel('Sample Size (n)')
    axes[1].set_ylabel('Mean Absolute Error')
    axes[1].set_title('MLE Convergence: Std Dev')
    axes[1].legend()
    axes[1].grid(True, alpha=0.2)

    plt.suptitle('MLE Convergence Rate', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()

    print('MLE converges at rate O(1/√n) — doubling precision needs 4× more data.')


analyze_mle_convergence()

### 4.2 Hypothesis Testing for Model ComparisonIn ML, we often need to determine if one model is significantly betterthan another. The paired t-test compares two sets of scores.

In [ ]:
def demonstrate_hypothesis_testing() -> None:
    """Show hypothesis testing for model comparison."""
    rng = np.random.RandomState(SEED)

    # Simulate cross-validation scores for two models
    n_folds = 30
    model_a_scores = rng.normal(0.85, 0.03, n_folds)  # Mean 85%
    model_b_scores = rng.normal(0.87, 0.03, n_folds)  # Mean 87%
    model_c_scores = rng.normal(0.855, 0.03, n_folds)  # Mean 85.5% (not significant)

    # Paired t-test: A vs B
    t_stat_ab, p_val_ab = sp_stats.ttest_rel(model_a_scores, model_b_scores)
    t_stat_ac, p_val_ac = sp_stats.ttest_rel(model_a_scores, model_c_scores)

    print('=== Model Comparison via Paired t-test ===')
    results = pd.DataFrame({
        'Comparison': ['A vs B', 'A vs C'],
        'Mean A': [f'{model_a_scores.mean():.4f}', f'{model_a_scores.mean():.4f}'],
        'Mean Other': [f'{model_b_scores.mean():.4f}', f'{model_c_scores.mean():.4f}'],
        'Diff': [f'{(model_b_scores - model_a_scores).mean():.4f}',
                 f'{(model_c_scores - model_a_scores).mean():.4f}'],
        't-stat': [f'{t_stat_ab:.3f}', f'{t_stat_ac:.3f}'],
        'p-value': [f'{p_val_ab:.4f}', f'{p_val_ac:.4f}'],
        'Significant?': [
            'Yes' if p_val_ab < 0.05 else 'No',
            'Yes' if p_val_ac < 0.05 else 'No',
        ],
    })
    print(results.to_string(index=False))
    print()

    # Visualize
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))

    # Box plot comparison
    bp = axes[0].boxplot([model_a_scores, model_b_scores, model_c_scores],
                          labels=['Model A', 'Model B', 'Model C'], patch_artist=True)
    for patch, color in zip(bp['boxes'], [COLORS['blue'], COLORS['green'], COLORS['orange']]):
        patch.set_facecolor(color)
        patch.set_alpha(0.6)
    axes[0].set_ylabel('Accuracy')
    axes[0].set_title('Cross-Validation Score Distributions')
    axes[0].grid(True, axis='y', alpha=0.2)

    # Difference distribution
    diff_ab = model_b_scores - model_a_scores
    diff_ac = model_c_scores - model_a_scores
    axes[1].hist(diff_ab, bins=15, color=COLORS['green'], alpha=0.6, label='B - A', edgecolor='white')
    axes[1].hist(diff_ac, bins=15, color=COLORS['orange'], alpha=0.6, label='C - A', edgecolor='white')
    axes[1].axvline(0, color='black', linestyle='--', alpha=0.5)
    axes[1].set_xlabel('Score Difference')
    axes[1].set_ylabel('Count')
    axes[1].set_title('Score Differences')
    axes[1].legend()

    plt.tight_layout()
    plt.show()

    print('Interpretation:')
    print('  p < 0.05 → reject null hypothesis (models perform differently)')
    print('  p > 0.05 → cannot reject null (no significant difference)')


demonstrate_hypothesis_testing()

### 4.3 Error Analysis: Common Probabilistic Mistakes in MLLet's examine common mistakes when applying probability in ML.

In [ ]:
def demonstrate_common_mistakes() -> None:
    """Show common probabilistic mistakes and their consequences."""
    rng = np.random.RandomState(SEED)

    print('=== Common Probabilistic Mistakes in ML ===')
    print()

    # 1. Confusing P(A|B) with P(B|A) — base rate fallacy
    print('1. BASE RATE FALLACY: P(disease|positive) ≠ P(positive|disease)')
    sensitivity = 0.99  # P(positive|disease)
    prevalence = 0.001  # P(disease)
    false_positive_rate = 0.05  # P(positive|healthy)

    p_positive = sensitivity * prevalence + false_positive_rate * (1 - prevalence)
    p_disease_given_positive = (sensitivity * prevalence) / p_positive

    print(f'  P(positive|disease) = {sensitivity:.0%} (sensitivity)')
    print(f'  P(disease) = {prevalence:.1%} (prevalence)')
    print(f'  P(positive|healthy) = {false_positive_rate:.0%} (false positive rate)')
    print(f'  P(disease|positive) = {p_disease_given_positive:.1%} (much lower than 99%!)')
    print()

    # 2. Multiple comparisons problem
    print('2. MULTIPLE COMPARISONS: Testing many models inflates false positives')
    n_models = 20
    p_values = rng.uniform(0, 1, n_models)  # All models are equally bad
    significant = np.sum(p_values < 0.05)
    print(f'  {n_models} random models tested at α=0.05')
    print(f'  "Significant" results: {significant} (expected ≈ {n_models * 0.05:.0f})')
    print(f'  Bonferroni correction: α = 0.05/{n_models} = {0.05/n_models:.4f}')
    bonferroni_sig = np.sum(p_values < 0.05 / n_models)
    print(f'  Significant after correction: {bonferroni_sig}')
    print()

    # 3. Independence assumption
    print('3. INDEPENDENCE ASSUMPTION: Most real features are not independent')
    corr_target = compute_correlation_from_scratch(
        np.column_stack([X_housing[:, 0], X_housing[:, 5], y_housing])
    )
    print(f'  Corr(MedInc, AveOccup) = {corr_target[0, 1]:.4f}')
    print(f'  Naive Bayes assumes independence — works despite this!')
    print()

    # Summary table
    mistakes = pd.DataFrame({
        'Mistake': [
            'Base rate fallacy',
            'Multiple comparisons',
            'Independence assumption',
            'Small sample inference',
            'Ignoring priors',
        ],
        'Description': [
            'Confusing P(A|B) with P(B|A)',
            'Testing many hypotheses without correction',
            'Assuming features are independent',
            'Drawing conclusions from n < 30',
            'Using flat priors when domain knowledge exists',
        ],
        'ML Impact': [
            'Misinterpreting classifier outputs',
            'Overfitting to validation set',
            'Naive Bayes, feature selection bias',
            'Unreliable cross-validation scores',
            'Missing regularization opportunity',
        ],
    })
    print('=== Summary ===')
    print(mistakes.to_string(index=False))


demonstrate_common_mistakes()

### 4.4 Probability & Statistics Reference for MLA comprehensive reference mapping probability concepts to their ML applications.

In [ ]:
def build_probability_reference() -> None:
    """Create a probability/statistics reference for the course."""
    reference = pd.DataFrame({
        'Concept': [
            'Gaussian distribution', 'Bernoulli/Categorical',
            'Maximum Likelihood', 'MAP estimation',
            'Bayes theorem', 'Central Limit Theorem',
            'Covariance/Correlation', 'Confidence intervals',
            'Hypothesis testing', 'KL divergence',
        ],
        'ML Application': [
            'Weight initialization, noise models',
            'Classification outputs, sampling',
            'Training = maximizing likelihood',
            'Regularization = adding a prior',
            'Posterior estimation, Naive Bayes',
            'Why batch means are Gaussian',
            'Feature selection, PCA (Module 1-06)',
            'Uncertainty quantification',
            'Model comparison, A/B testing',
            'Loss functions, VAEs (see 01-08)',
        ],
        'Where Used': [
            'Modules 1-20', 'Modules 2, 5, 7, 10',
            'Modules 2-20', 'Modules 2, 4, 13',
            'Modules 2, 4, 7', 'Modules 4, 16',
            'Modules 1, 3, 4', 'Modules 4, 20',
            'Modules 4, 20', 'Modules 1, 5, 11',
        ],
    })
    print('=== Probability & Statistics for ML — Reference ===')
    print(reference.to_string(index=False))


build_probability_reference()

---## Part 5 — Summary & Lessons Learned### Key Takeaways1. **Training = Maximum Likelihood.** Minimizing cross-entropy loss is exactly   maximizing the log-likelihood of the data under the model. Understanding   this connection demystifies why loss functions look the way they do.2. **Regularization = Prior belief.** MAP estimation adds a prior to MLE.   Gaussian prior → L2 (Ridge), Laplace prior → L1 (Lasso). More data   reduces the influence of the prior.3. **Bayes' theorem updates beliefs.** The posterior combines prior knowledge   with evidence. This is the foundation of Bayesian methods, Naive Bayes   classification, and probabilistic graphical models.4. **The CLT explains Gaussian ubiquity.** Sample means converge to Gaussians   regardless of the source distribution. This justifies many Gaussian   assumptions in ML (batch statistics, noise models, initialization).5. **Statistical testing prevents false discoveries.** Use paired t-tests   for model comparison, apply Bonferroni correction for multiple tests,   and be aware of base rate fallacies when interpreting probabilities.### What's Next→ **01-08 (Information Theory for ML)** covers entropy, cross-entropy,  KL divergence, and mutual information — the information-theoretic  perspective on the same loss functions we derived probabilistically here.### Going Further- [Pattern Recognition and Machine Learning (Bishop)](https://www.microsoft.com/en-us/research/publication/pattern-recognition-machine-learning/) —  The classic probabilistic ML textbook- [Probabilistic Machine Learning (Murphy)](https://probml.github.io/pml-book/) —  Modern treatment with deep learning connections- [Seeing Theory](https://seeing-theory.brown.edu/) — Interactive probability  visualizations